In [1]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000 -report

# import urllib.request

# url = "https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv"
# filename = "hdma-wi-2021.csv"

# urllib.request.urlretrieve(url, filename)

# # # Remove the files if they already exist in HDFS
# # !hdfs dfs -rm -f hdfs://boss:9000/single.csv
# # !hdfs dfs -rm -f hdfs://boss:9000/double.csv

# # !HADOOP_user=hdfs hdfs dfs -D dfs.replication=1 -put hdma-wi-2021.csv hdfs://boss:9000/single.csv
# # !HADOOP_user=hdfs hdfs dfs -D dfs.replication=2 -put hdma-wi-2021.csv hdfs://boss:9000/double.csv

# Some of the source codes are from Microsoft Bing

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 37011935513 (34.47 GB)
DFS Remaining: 36481888256 (33.98 GB)
DFS Used: 530047257 (505.49 MB)
DFS Used%: 1.43%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.18.0.4:9866 (project-4-junho_4-dn-2.project-4-junho_4_default)
Hostname: 33e70ca9cb82
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 265998336 (253.68 MB)
Non DFS Used: 7297355776 (6.80 GB)
DFS Remaining: 18240921600 (16.99 GB)
DFS Used%: 1.03%
DFS Remaining

In [ ]:
import time
time.sleep(30)

In [3]:
#q9
import requests
import json

# WebHDFS URL for the file
# From OH
url = "http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS"
# Send a GET request
response = requests.get(url)

# Convert the response to JSON
data = response.json()
data

redirect_url = data['BlockLocations']['BlockLocation'] # same idea from #5

#block_distribution = {}

#redirect_url
#Initialize an empty dictionary to store the block distribution
block_distribution = {}
data['BlockLocations']['BlockLocation']
# Loop through each block (from Bing)
for block in data['BlockLocations']['BlockLocation']:
    # Extract the container ID from the hosts
    if(block['hosts'] != []):
        container_id = block['hosts'][0]
    else:
        container_id = 'lost'
    
    if container_id in block_distribution:
        block_distribution[container_id] += 1
    else:
        block_distribution[container_id] = 1

# # Print the block distribution
block_distribution

{'lost': 83, '33e70ca9cb82': 84}

In [4]:
#q10
import pyarrow as pa
import pyarrow.fs

# Connect to HDFS
hdfs = pa.fs.HadoopFileSystem("boss", 9000)

url = "http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS"

# Send a GET request
response = requests.get(url)

# Convert the response to JSON
data = response.json()

# Initialize the count of "Single Family"
count = 0

# Loop through each block
for block in data['BlockLocations']['BlockLocation']:
    # Extract the container ID from the hosts
    if block["hosts"] != []:
        # Open the file
        with hdfs.open_input_file("hdfs://boss:9000/single.csv") as file:
            # Read the first 10 bytes of the file
            bytes = file.read_at(block["length"], block["offset"])
            count += str(bytes).count("Single Family")

count

2023-10-27 23:34:36,114 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


223857